In [101]:
# install packages
!pip install inflect

In [135]:
!pip install vaderSentiment

In [2]:
!pip install textblob

In [110]:
!pip install flair

  Using cached flair-0.6.1.post1-py3-none-any.whl (337 kB)
  Using cached bpemb-0.3.2-py3-none-any.whl (19 kB)
  Using cached langdetect-1.0.8.tar.gz (981 kB)
  Using cached hyperopt-0.2.5-py2.py3-none-any.whl (965 kB)
  Using cached segtok-1.5.10.tar.gz (25 kB)
  Using cached tabulate-0.8.7-py3-none-any.whl (24 kB)
  Using cached konoha-4.6.2-py3-none-any.whl (19 kB)
  Using cached sentencepiece-0.1.94-cp38-cp38-win_amd64.whl (1.2 MB)
  Using cached mpld3-0.3.tar.gz (788 kB)
  Using cached gdown-3.12.2.tar.gz (8.2 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached ftfy-5.8.tar.gz (64 kB)
  Using cached sqlitedict-1.7.0.tar.gz (28 kB)
  Using cached transformers-3.5.1-py3-none-any.whl (1.3 MB)
  Using cached Ja

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

transformers 3.5.1 requires sentencepiece==0.1.91, but you'll have sentencepiece 0.1.94 which is incompatible.


  Attempting uninstall: mpld3
    Found existing installation: mpld3 0.5.1
    Uninstalling mpld3-0.5.1:
      Successfully uninstalled mpld3-0.5.1


In [128]:
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in c:\users\kevinzhenshuai\anaconda3\lib\site-packages (2.3.1)


In [130]:
pip install --upgrade keras

Requirement already up-to-date: keras in c:\users\kevinzhenshuai\anaconda3\lib\site-packages (2.4.3)Note: you may need to restart the kernel to use updated packages.



## Import necessary libraries

In [2]:
import json
import pandas as pd
import string
import re
import time
from datetime import datetime, timezone
import numpy as np

import nltk
import html
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from flair.embeddings import FlairEmbeddings, BertEmbeddings, WordEmbeddings, DocumentRNNEmbeddings

import tensorflow as tf
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import SpatialDropout1D, LSTM
from tensorflow.keras import regularizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Reshape
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.convolutional import MaxPooling1D, MaxPooling2D

from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from scipy.sparse import hstack, csr_matrix, coo_matrix
from sklearn.model_selection import train_test_split

from textblob import TextBlob

from keras.layers import Input, Embedding, Dot, Reshape, Dense
from keras.models import Model

## Import data files

Training Data

In [3]:
trainList = []
print("Started Reading JSON file which contains multiple JSON document")
with open("train.json", "r") as f:
    for jsonObj in f:
        trainDict = json.loads(jsonObj)
        trainList.append(trainDict)
        
trainDF = pd.DataFrame(data = trainList)
trainDF.info()
trainDF.head()

Started Reading JSON file which contains multiple JSON document
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         200000 non-null  float64
 1   reviewTime      200000 non-null  object 
 2   reviewerID      200000 non-null  object 
 3   reviewText      199964 non-null  object 
 4   summary         199965 non-null  object 
 5   unixReviewTime  200000 non-null  int64  
 6   category        200000 non-null  object 
 7   price           200000 non-null  object 
 8   itemID          200000 non-null  object 
 9   reviewHash      200000 non-null  object 
 10  image           463 non-null     object 
dtypes: float64(1), int64(1), object(9)
memory usage: 16.8+ MB


,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,image
0,4.0,"08 24, 2010",u04428712,"So is Katy Perry's new album ""Teenage Dream"" c...",Amazing that I Actually Bought This...More Ama...,1282608000,Pop,$35.93,p70761125,85559980,NaN
1,5.0,"10 31, 2009",u06946603,"I got this CD almost 10 years ago, and given t...",Excellent album,1256947200,Alternative Rock,$11.28,p85427891,41699565,NaN
2,4.0,"10 13, 2015",u92735614,I REALLY enjoy this pairing of Anderson and Po...,"Love the Music, Hate the Light Show",1444694400,Pop,$89.86,p82172532,24751194,NaN
3,5.0,"06 28, 2017",u35112935,Finally got it . It was everything thought it ...,Great,1498608000,Pop,$11.89,p15255251,22820631,NaN
4,4.0,"10 12, 2015",u07141505,"Look at all star cast. Outstanding record, pl...",Love these guys.,1444608000,Jazz,$15.24,p82618188,53377470,NaN


Test Data

In [3]:
testList = []
print("Started Reading JSON file which contains multiple JSON document")
with open("test.json", "r") as f:
    for jsonObj in f:
        testDict = json.loads(jsonObj)
        testList.append(testDict)
        
testDF = pd.DataFrame(data = testList)
testDF.info()
testDF.head()

Started Reading JSON file which contains multiple JSON document
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewTime      10000 non-null  object
 1   reviewerID      10000 non-null  object
 2   reviewText      9999 non-null   object
 3   summary         9998 non-null   object
 4   unixReviewTime  10000 non-null  int64 
 5   category        10000 non-null  object
 6   price           10000 non-null  object
 7   itemID          10000 non-null  object
 8   reviewHash      10000 non-null  object
 9   image           23 non-null     object
dtypes: int64(1), object(9)
memory usage: 781.4+ KB


,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,image
0,"03 26, 2015",u32476110,"Fantastic mix of ""old school"" with a creative ...","Fantastic mix of ""old school"" with a creative ...",1427328000,Pop,$14.98,p76243483,20167847,NaN
1,"05 15, 2017",u36732410,Update: Indications\nThere are various opinion...,Digitally Extracted Stereo (DES) Rules!,1494806400,Pop,$15.16,p92485419,30527605,NaN
2,"06 4, 2015",u85385007,This album provides a new twist on old Sammy H...,Excellent unplugged album,1433376000,Pop,$7.37,p40031588,12169432,NaN
3,"04 23, 2009",u30715529,(Symbol) can be considered as another masterpi...,another masterpiece,1240444800,Pop,$12.45,p88719785,55648615,NaN
4,"09 15, 2000",u95909892,Many would think this album is good only becau...,True Classic Rock,968976000,Alternative Rock,$2.07,p59188380,09520938,NaN


## Data Cleaning and Preparation

Denoise Text

In [4]:
def html_escaping(text):
    '''html escaping: Html character codes (i.e., &...;) are replaced with an ASCII equivalent'''
    return html.unescape(text)

def to_lower(text):
    '''lower case all words in text'''
    return text.lower()

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_url(text):
    '''remove urls'''
    return re.sub(r"http\S+", ' ', text)

def remove_punctuation(text):
    '''Remove punctuation from text'''
    return re.sub(r'[^\w\s]', '', text)
        
def denoise_text(text):
    text = str(text) # make sure all inputs are type string
    text = to_lower(text)
    text = strip_html(text)
    text = html_escaping(text)
    text = remove_between_square_brackets(text)
    text = remove_punctuation(text)
    return text

Tokenize and normalize text

In [5]:
def word_extraction(text):    
    '''removing stopwords from the sentence and tokenize the sentence'''
    stopWords = stopwords.words('english')
    # append words with puncutations stripped
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in stopWords]
    stopWords.extend(stripped)
    # remove duplicates for runtime
    stopWords= list(dict.fromkeys(stopWords))
    
    ignore = stopWords   
    words = re.sub("[^\w]", " ",  text).split()    
    cleaned_words = [w for w in words if w not in ignore]    
    return cleaned_words

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def detokenize_text(words):
    '''detokenize words into a text string'''
    return ' '.join(words)

def normalize_text(text):
    words = word_extraction(text)
    #words = lemmatize_verbs(words)
    words = stem_words(words)
    text = detokenize_text(words)
    return text

### Categorical Data

#### unixReviewTime

In [6]:
def get_weekday(unixTime):
    '''Extact Weekday as locale’s abbreviated name from unixReviewTime as the feature'''
    return datetime.fromtimestamp(unixTime, timezone.utc).strftime("%A")

def get_year(unixTime):
    return int(datetime.fromtimestamp(unixTime,timezone.utc).strftime('%Y'))

In [7]:
#TRAIN -------------------------------------------------------
trainDF['weekday'] = trainDF['unixReviewTime'].apply(lambda x: get_weekday(x))
trainDF['year'] = trainDF['unixReviewTime'].apply(lambda x: get_year(x))

In [8]:
#TEST -------------------------------------------------------
testDF['weekday'] = testDF['unixReviewTime'].apply(lambda x: get_weekday(x))
testDF['year'] = testDF['unixReviewTime'].apply(lambda x: get_year(x))

In [9]:
trainDF['weekday'].value_counts()

Tuesday      32180
Wednesday    29868
Monday       29066
Thursday     28796
Friday       28243
Saturday     26427
Sunday       25420
Name: weekday, dtype: int64

In [10]:
testDF['weekday'].value_counts()

Tuesday      1655
Wednesday    1524
Friday       1433
Thursday     1413
Monday       1412
Saturday     1316
Sunday       1247
Name: weekday, dtype: int64

#### price

training data

In [11]:
trainDF['price'] = trainDF['price'].str.replace('$', '')
trainDF['price'] = trainDF['price'].apply(lambda x : pd.to_numeric(x,errors = 'coerce'))

In [12]:
# There are several methods to determine how to handle NaN values 
#- including dropping them, filling with mode or mean, or replacing with an "Other" value
# start with data exploration - can check if there a pattern in where the NaN values appear or if the rows with NaN are randomly distributed

# For Example - is the a pattern in job categories where NaN contract type appears or is it similar to the full data set?

a = trainDF['category'].value_counts(normalize=True)
b = trainDF[trainDF['price'].isnull()]['category'].value_counts(normalize=True)
print (pd.DataFrame({'Alldata': a, 'NaN Price':b,'ratio':b/a}))

                     Alldata  NaN Price     ratio
Alternative Rock    0.283790   0.357012  1.258015
Classical           0.094275   0.082754  0.877789
Dance & Electronic  0.062630   0.050165  0.800970
Jazz                0.098930   0.072867  0.736552
Pop                 0.460375   0.437202  0.949666


In [13]:
# fill null price with mode
trainDF['price'].fillna(trainDF['price'].mode()[0],inplace=True)

test data

In [14]:
testDF['price'] = testDF['price'].str.replace('$', '')
testDF['price'] = testDF['price'].apply(lambda x : pd.to_numeric(x,errors = 'coerce'))

In [15]:
a = testDF['category'].value_counts(normalize=True)
b = testDF[testDF['price'].isnull()]['category'].value_counts(normalize=True)
print (pd.DataFrame({'Alldata': a, 'NaN Price':b,'ratio':b/a}))

                    Alldata  NaN Price     ratio
Alternative Rock     0.2901   0.349650  1.205275
Classical            0.0903   0.083916  0.929303
Dance & Electronic   0.0627   0.083916  1.338375
Jazz                 0.0993   0.076923  0.774653
Pop                  0.4576   0.405594  0.886351


In [16]:
# fill null price with mode
testDF['price'].fillna(testDF['price'].mode()[0],inplace=True)

#### category

In [17]:
trainDF.category.value_counts()

Pop                   92075
Alternative Rock      56758
Jazz                  19786
Classical             18855
Dance & Electronic    12526
Name: category, dtype: int64

In [18]:
review_counts = trainDF.reviewerID.value_counts()
value_mask = trainDF.reviewerID.isin(review_counts.index[review_counts < 20])
trainDF.loc[value_mask,'reviewerID'] = "Other"
testDF.loc[testDF.reviewerID.isin(list(review_counts.index[review_counts > 20])) == False,\
                  "reviewerID"] = "Other"

In [19]:
trainDF = pd.get_dummies(data=trainDF, columns=["reviewerID",'weekday','category'])
testDF = pd.get_dummies(data=testDF, columns=["reviewerID",'weekday','category'])


# In case there are missing columns in Test
missing_cols = set( trainDF.columns ) - set(testDF.columns )
for column in missing_cols:
    testDF[column] = 0
testDF = testDF[trainDF.columns]

### NLP on text data

#### reviewText

In [20]:
%%time

#TRAIN -------------------------------------------------------
trainDF['reviewText'] = trainDF['reviewText'].apply(lambda x: denoise_text(x)) 
trainDF['reviewText'] = trainDF['reviewText'].apply(lambda x: normalize_text(x)) 
trainDF['reviewText'].head()

C:\Users\kevinzhenshuai\anaconda3\lib\site-packages\bs4\__init__.py:414: MarkupResemblesLocatorWarning: "http://www.amazon.com/gp/product/b003hjwjy6?redirect=true&ref_=cm_cr_ryp_prd_ttl_sol_2" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


Wall time: 5min 51s


0    katy perry new alb teen dream cont der unorigi...
1    got cd almost 10 year ago giv pass tim stil th...
2    real enjoy pair anderson ponty mus much would ...
3                fin got everyth thought would gre mus
4    look star cast outstand record play impress 4 ...
Name: reviewText, dtype: object

In [21]:
trainDF['reviewText'] = trainDF['reviewText'].apply(lambda x: 'unknown' if x == '' else x)

In [22]:
trainDF.loc[trainDF['reviewText'] == 'unknown'] 

,overall,reviewTime,reviewText,summary,unixReviewTime,price,itemID,reviewHash,image,year,...,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,category_Alternative Rock,category_Classical,category_Dance & Electronic,category_Jazz,category_Pop
542,5.0,"05 3, 2018",unknown,Five Stars,1525305600,14.99,p51175958,77435483,NaN,2018,...,0,0,1,0,0,0,0,0,0,1
876,4.0,"12 23, 2014",unknown,Four Stars,1419292800,18.82,p62233954,47305192,NaN,2014,...,0,0,0,1,0,1,0,0,0,0
923,5.0,"02 26, 2015",unknown,Five Stars,1424908800,12.98,p36840459,82555669,NaN,2015,...,0,0,1,0,0,0,0,0,1,0
5411,3.0,"04 24, 2018",unknown,Three Stars,1524528000,11.99,p12318999,62207941,NaN,2018,...,0,0,0,1,0,0,0,0,0,1
7409,4.0,"06 1, 2017",unknown,Classic Tina...,1496275200,14.57,p24847185,74826717,NaN,2017,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196797,5.0,"07 9, 2014",unknown,Five Stars,1404864000,10.04,p01993249,68055384,NaN,2014,...,0,0,0,0,1,1,0,0,0,0
198588,5.0,"01 12, 2016",unknown,Five Stars,1452556800,18.98,p41783190,38135696,NaN,2016,...,0,0,0,1,0,0,1,0,0,0
198606,5.0,"02 9, 2017",unknown,Five Stars,1486598400,13.98,p23085153,57943989,NaN,2017,...,0,0,1,0,0,0,0,0,0,1
198630,5.0,"09 28, 2017",unknown,Five Stars,1506556800,14.39,p98744367,68660574,NaN,2017,...,0,0,1,0,0,0,1,0,0,0


In [23]:
%%time

#TEST -------------------------------------------------------
testDF['reviewText'] = testDF['reviewText'].apply(lambda x: denoise_text(x)) 
testDF['reviewText'] = testDF['reviewText'].apply(lambda x: normalize_text(x)) 
testDF.head()

C:\Users\kevinzhenshuai\anaconda3\lib\site-packages\bs4\__init__.py:414: MarkupResemblesLocatorWarning: "http://www.amazon.com/jackrabbit-san-fermin/dp/b00tbbaryi?ie=utf8&redirect=true&ref_=cm_cr_ryp_prd_ttl_sol_9" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


Wall time: 17.1 s


,overall,reviewTime,reviewText,summary,unixReviewTime,price,itemID,reviewHash,image,year,...,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,category_Alternative Rock,category_Classical,category_Dance & Electronic,category_Jazz,category_Pop
0,0,"03 26, 2015",fantast mix old school cre new sound heard sim...,"Fantastic mix of ""old school"" with a creative ...",1427328000,14.98,p76243483,20167847,NaN,2015,...,0,0,1,0,0,0,0,0,0,1
1,0,"05 15, 2017",upd ind vary opin regard mono vers stereo bott...,Digitally Extracted Stereo (DES) Rules!,1494806400,15.16,p92485419,30527605,NaN,2017,...,0,0,0,0,0,0,0,0,0,1
2,0,"06 4, 2015",alb provid new twist old sammy hag van hal son...,Excellent unplugged album,1433376000,7.37,p40031588,12169432,NaN,2015,...,0,0,1,0,0,0,0,0,0,1
3,0,"04 23, 2009",symbol consid anoth masterpiec print put fanta...,another masterpiece,1240444800,12.45,p88719785,55648615,NaN,2009,...,0,0,1,0,0,0,0,0,0,1
4,0,"09 15, 2000",many would think alb good get would wrong asse...,True Classic Rock,968976000,2.07,p59188380,09520938,NaN,2000,...,0,0,0,0,0,1,0,0,0,0


In [24]:
testDF['reviewText'] = testDF['reviewText'].apply(lambda x: 'unknown' if x == '' else x)

In [25]:
testDF.loc[testDF['reviewText'] == 'unknown'] 

,overall,reviewTime,reviewText,summary,unixReviewTime,price,itemID,reviewHash,image,year,...,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,category_Alternative Rock,category_Classical,category_Dance & Electronic,category_Jazz,category_Pop
132,0,"01 30, 2017",unknown,Five Stars,1485734400,11.94,p84457806,59392884,NaN,2017,...,0,0,0,0,0,0,0,0,1,0
448,0,"06 13, 2016",unknown,Five Stars,1465776000,9.68,p51656766,03839526,NaN,2016,...,0,0,0,0,0,0,0,0,0,1
799,0,"05 3, 2017",unknown,Five Stars,1493769600,15.16,p92485419,46674136,NaN,2017,...,0,0,0,0,1,0,0,0,0,1
983,0,"08 25, 2003",unknown,"No, No, No, etc.",1061769600,8.98,p90041047,26513936,NaN,2003,...,0,0,0,0,0,0,0,0,0,1
1455,0,"04 10, 2015",unknown,Five Stars,1428624000,30.75,p53702411,12005050,NaN,2015,...,0,0,0,0,0,0,0,0,1,0
1996,0,"10 12, 2015",unknown,Five Stars,1444608000,4.11,p33589931,02044535,NaN,2015,...,0,0,0,0,0,0,0,0,0,1
2144,0,"04 29, 2015",unknown,Five Stars,1430265600,11.66,p44060712,07344599,NaN,2015,...,0,0,0,0,1,1,0,0,0,0
3820,0,"08 4, 2014",unknown,Great album from Anberlin,1407110400,14.90,p03021942,40666447,NaN,2014,...,0,0,0,0,0,1,0,0,0,0
3839,0,"04 19, 2015",unknown,Five Stars,1429401600,16.76,p18504647,67902365,NaN,2015,...,0,1,0,0,0,1,0,0,0,0
4122,0,"09 9, 2015",unknown,Five Stars,1441756800,13.40,p31593118,35395914,NaN,2015,...,0,0,0,0,1,0,0,0,0,1


#### Summary

In [26]:
%%time

#TRAIN -------------------------------------------------------
trainDF['summary'] = trainDF['summary'].apply(lambda x: denoise_text(x)) 
trainDF['summary'] = trainDF['summary'].apply(lambda x: normalize_text(x))
trainDF['summary'].head()

C:\Users\kevinzhenshuai\anaconda3\lib\site-packages\bs4\__init__.py:329: MarkupResemblesLocatorWarning: "..." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\kevinzhenshuai\anaconda3\lib\site-packages\bs4\__init__.py:329: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\kevinzhenshuai\anaconda3\lib\site-packages\bs4\__init__.py:329: MarkupResemblesLocatorWarning: "......." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\kevinzhenshuai\anaconda3\lib\site-packages\bs4\__init__.py:329: MarkupResemblesLocatorWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\kevinzhenshuai\anacond

Wall time: 1min 47s


0    amaz act bought thism amaz act lov
1                             excel alb
2                lov mus hat light show
3                                   gre
4                               lov guy
Name: summary, dtype: object

In [27]:
trainDF['summary'] = trainDF['summary'].apply(lambda x: 'unknown' if x == '' else x)

In [28]:
trainDF.loc[trainDF['summary'] == 'unknown'] 

,overall,reviewTime,reviewText,summary,unixReviewTime,price,itemID,reviewHash,image,year,...,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,category_Alternative Rock,category_Classical,category_Dance & Electronic,category_Jazz,category_Pop
520,2.0,"05 11, 2016",mus liv expect,unknown,1462924800,13.98,p37967807,28328461,NaN,2016,...,0,0,0,0,1,0,0,0,0,1
675,3.0,"12 1, 2008",say tta us voc cre structured song mislead imm...,unknown,1228089600,24.99,p55218454,94891540,NaN,2008,...,0,0,0,0,0,0,0,1,0,0
1462,1.0,"11 30, 2011",hat record first bought 1994 stil hat lik list...,unknown,1322611200,10.28,p19498936,31739644,NaN,2011,...,0,0,0,0,1,0,0,0,0,1
1666,5.0,"01 27, 2013",quick acur could ask anyth would go anywh els ...,unknown,1359244800,11.67,p96637662,72562076,NaN,2013,...,0,1,0,0,0,0,0,0,0,1
2017,1.0,"01 21, 2003",cd desecr wond class stick know go back rock rol,unknown,1043107200,6.99,p12955004,83698948,NaN,2003,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199329,3.0,"03 16, 2000",blar guit blurt strange lyr limp bizkit toa me...,unknown,953164800,8.49,p92745515,54391122,NaN,2000,...,0,0,1,0,0,1,0,0,0,0
199472,5.0,"08 5, 2015",ev kind lik jeff beck list ev big fan,unknown,1438732800,15.16,p89761415,76911336,NaN,2015,...,0,0,0,0,1,0,0,0,0,1
199512,3.0,"02 25, 2006",da su se sci pojavil pre odlicnih bravery soli...,unknown,1140825600,13.98,p92284858,16002913,NaN,2006,...,1,0,0,0,0,1,0,0,0,0
199858,5.0,"01 24, 2011",beat ant bring heat across country sint start ...,unknown,1295827200,12.99,p72706277,53922793,NaN,2011,...,0,0,0,0,0,0,0,1,0,0


In [29]:
%%time

#TEST -------------------------------------------------------
testDF['summary'] = testDF['summary'].apply(lambda x: denoise_text(x)) 
testDF['summary'] = testDF['summary'].apply(lambda x: normalize_text(x)) 

C:\Users\kevinzhenshuai\anaconda3\lib\site-packages\bs4\__init__.py:329: MarkupResemblesLocatorWarning: "..." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\kevinzhenshuai\anaconda3\lib\site-packages\bs4\__init__.py:329: MarkupResemblesLocatorWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\kevinzhenshuai\anaconda3\lib\site-packages\bs4\__init__.py:329: MarkupResemblesLocatorWarning: ".................." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Wall time: 5.19 s


In [30]:
testDF['summary'] = testDF['summary'].apply(lambda x: 'unknown' if x == '' else x)

In [31]:
testDF.loc[testDF['summary'] == 'unknown'] 

,overall,reviewTime,reviewText,summary,unixReviewTime,price,itemID,reviewHash,image,year,...,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,category_Alternative Rock,category_Classical,category_Dance & Electronic,category_Jazz,category_Pop
386,0,"09 29, 2005",stop review sig ros album soon get otherw woul...,unknown,1127952000,15.45,p75868193,65801856,NaN,2005,...,0,0,1,0,0,1,0,0,0,0
474,0,"03 23, 2006",nin hag good voc on second sing beautifu next ...,unknown,1143072000,6.14,p26406868,08359673,NaN,2006,...,0,0,1,0,0,1,0,0,0,0
813,0,"03 17, 2005",along almost every teen know us self titl cd o...,unknown,1111017600,8.32,p72412766,92232441,NaN,2005,...,0,0,1,0,0,1,0,0,0,0
1493,0,"11 12, 2006",put hop dream fergy fun party alb look sery mu...,unknown,1163289600,5.49,p48238587,90920112,NaN,2006,...,0,1,0,0,0,0,0,0,0,1
1773,0,"10 26, 2009",sad near four month sint trag heartbreak dea m...,unknown,1256515200,12.45,p22604624,20310190,NaN,2009,...,0,0,0,0,0,0,0,0,0,1
3366,0,"10 26, 2004",strokes recommend stud on class teach enjoy ro...,unknown,1098748800,37.99,p22016491,89076321,NaN,2004,...,0,0,0,1,0,1,0,0,0,0
3925,0,"08 31, 2009",slight disappoint cd guess expect someth gre l...,unknown,1251676800,5.00,p59697436,82712499,NaN,2009,...,0,0,0,0,0,0,0,0,0,1
4330,0,"06 15, 2004",fak get at wom real rock med afraid instead ge...,unknown,1087257600,4.66,p18498038,06613421,NaN,2004,...,0,0,0,1,0,0,0,0,0,1
4386,0,"11 2, 2000",origin pick cd almost year ago got sign mojo a...,unknown,973123200,4.70,p61312506,52736405,NaN,2000,...,0,0,1,0,0,1,0,0,0,0
4594,0,"01 21, 2015",gre,unknown,1421798400,10.19,p01434185,46118344,NaN,2015,...,0,0,0,0,1,0,0,0,0,1


### Try Sentiment Scoring with Flair, VADER, and textblob

In [32]:
# class FlairSentiment:
#     """Predict fine-grained sentiment scores using Flair."""
#     def __init__(self, model_file: str=None) -> None:
#         super().__init__()
#         from flair.models import TextClassifier
#         self.model = TextClassifier.load(model_file)

#     def score(self, text: str) -> int:
#         from flair.data import Sentence
#         doc = Sentence(text)
#         self.model.predict(doc)

#         if doc.labels[0].value == 'POSITIVE':
#             pred = doc.labels[0].score
#         else:
#             pred = -doc.labels[0].score
#         return pred

# flair_sentiment = FlairSentiment('en-sentiment')


# trainDF['review_sentiment'] = trainDF['reviewText'].apply(lambda x: flair_sentiment.score(x)) #Wall time: 30min 18s
# testDF['review_sentiment'] = testDF['reviewText'].apply(lambda x: flair_sentiment.score(x)) #Wall time: 1min 30s
# trainDF['summary_sentiment'] = trainDF['summary'].apply(lambda x: flair_sentiment.score(x)) #Wall time: 19min 17s
# testDF['summary_sentiment'] = testDF['summary'].apply(lambda x: flair_sentiment.score(x))  #Wall time: 57.5 s

In [33]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
sentence = "The food was fine!" 
vs = analyzer.polarity_scores(sentence)
vs['compound']

0.2714

In [34]:
%%time 
trainDF['review_sentiment'] = trainDF['reviewText'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
testDF['review_sentiment'] = testDF['reviewText'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
trainDF['summary_sentiment'] = trainDF['summary'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
testDF['summary_sentiment'] = testDF['summary'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

Wall time: 3min 47s


### Textblob

In [35]:
# sentence = "The food was neutral!" 
# TextBlob(sentence).sentiment.polarity 

# %%time 
# trainDF['review_sentiment'] = trainDF['reviewText'].apply(lambda x:TextBlob(x).sentiment.polarity )
# testDF['review_sentiment'] = testDF['reviewText'].apply(lambda x:TextBlob(x).sentiment.polarity)
# trainDF['summary_sentiment'] = trainDF['summary'].apply(lambda x: TextBlob(x).sentiment.polarity)
# testDF['summary_sentiment'] = testDF['summary'].apply(lambda x:TextBlob(x).sentiment.polarity)

#### Further Split Training Data into Training and Validation Set

In [36]:
RRP_x_train, RRP_x_val, RRP_y_train, RRP_y_val = train_test_split(trainDF.drop(['overall'],axis = 1), trainDF['overall'] , random_state=42, test_size=0.3)

In [37]:
testDF = testDF.drop(['overall'],axis = 1)

### Feturization

Featurize

In [38]:
%%time
reviewText_vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, 
                                                preprocessor = None, 
                                                stop_words = None, 
                                                max_features = 2000,
                                                ngram_range = (1,2))

train_reviewText = reviewText_vectorizer.fit_transform(RRP_x_train['reviewText'])
val_reviewText = reviewText_vectorizer.transform(RRP_x_val['reviewText'])
test_reviewText = reviewText_vectorizer.transform(testDF['reviewText'])
print(reviewText_vectorizer.get_feature_names()[0:20])

#Combine all features into sparse dataframe

#TRAIN -------------------------------------------------------

# input feature dataframe 
reviewText_train =  pd.DataFrame(data = train_reviewText.toarray(), columns = reviewText_vectorizer.get_feature_names())

reviewText_train.reset_index(drop=True, inplace=True)

#VALIDATION -------------------------------------------------------

# input feature dataframe 
reviewText_val =  pd.DataFrame(data = val_reviewText.toarray(), columns = reviewText_vectorizer.get_feature_names())

reviewText_val.reset_index(drop=True, inplace=True)

#TEST -------------------------------------------------------
reviewText_test =  pd.DataFrame(data = test_reviewText.toarray(), columns = reviewText_vectorizer.get_feature_names())

reviewText_test.reset_index(drop=True, inplace=True)

['10', '100', '1010', '11', '12', '13', '14', '15', '16', '1st', '20', '25', '2nd', '30', '35', '3rd', '40', '45', '55', '60s']
Wall time: 28.1 s


In [39]:
summary_vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, 
                                                preprocessor = None, 
                                                stop_words = None, 
                                                max_features = 2000,
                                                ngram_range = (1,2))

train_summary = summary_vectorizer.fit_transform(RRP_x_train['summary'])
val_summary = summary_vectorizer.transform(RRP_x_val['summary'])
test_summary = summary_vectorizer.transform(testDF['summary'])

print(summary_vectorizer.get_feature_names()[0:20])

#Combine all features into sparse dataframe

#TRAIN -------------------------------------------------------

# input feature dataframe 
summary_train =  pd.DataFrame(data = train_summary.toarray(), columns = summary_vectorizer.get_feature_names())

summary_train.reset_index(drop=True, inplace=True)

#VALIDATION -------------------------------------------------------

# input feature dataframe 
summary_val =  pd.DataFrame(data = val_summary.toarray(), columns = summary_vectorizer.get_feature_names())

summary_val.reset_index(drop=True, inplace=True)

#TEST -------------------------------------------------------

summary_test =  pd.DataFrame(data = test_summary.toarray(), columns = summary_vectorizer.get_feature_names())

summary_test.reset_index(drop=True, inplace=True)

['10', '10 star', '100', '12', '12 star', '1st', '20', '20 year', '2004', '2005', '2013', '2014', '2015', '20th', '20th century', '25', '2nd', '35', '35 star', '3rd']


### Combine Features

In [40]:
# %%time
# #TRAIN -------------------------------------------------------
# RRP_x_train = hstack((csr_matrix(feature_train), 
#                      csr_matrix(trainDF[['price','review_sentiment','summary_sentiment']]),
#                      csr_matrix(summary_train), 
#                      csr_matrix(reviewText_train)))
# RRP_y_train =  trainDF['overall']

# #TEST -------------------------------------------------------
# RRP_x_test = hstack((csr_matrix(feature_test), 
#                      csr_matrix(testDF[['price','review_sentiment','summary_sentiment']]),
#                      csr_matrix(summary_test), 
#                      csr_matrix(reviewText_test)))


In [41]:
%%time
RRP_x_train['sentiment'] = (RRP_x_train['review_sentiment']+RRP_x_train['summary_sentiment'])/2
RRP_x_val['sentiment'] = (RRP_x_val['review_sentiment']+RRP_x_val['summary_sentiment'])/2
testDF['sentiment'] = (testDF['review_sentiment']+testDF['summary_sentiment'])/2

Wall time: 20 ms


In [42]:
# categoricalDF_train = RRP_x_train[['weekday','category']]
# feature_train = pd.get_dummies(data=categoricalDF_train, columns=['weekday','category'])

# categoricalDF_val = RRP_x_val[['weekday','category']]
# feature_val = pd.get_dummies(data=categoricalDF_val, columns=['weekday','category'])

# categoricalDF_test = testDF[['weekday','category']]
# feature_test = pd.get_dummies(data=categoricalDF_test, columns=['weekday','category'])

#TRAIN -------------------------------------------------------
feature_train = RRP_x_train.drop(['reviewTime','reviewText','summary','unixReviewTime','itemID','reviewHash','image'], axis=1)
#VALIDATION -------------------------------------------------------
feature_val = RRP_x_val.drop(['reviewTime','reviewText','summary','unixReviewTime','itemID','reviewHash','image'], axis=1)

#TEST -------------------------------------------------------
feature_test = testDF.drop(['reviewTime','reviewText','summary','unixReviewTime','itemID','reviewHash','image'], axis=1)

In [43]:
%%time
RRP_x_train = RRP_x_train.reset_index(drop=True)
summary_train = summary_train.reset_index(drop=True)
feature_train = feature_train.reset_index(drop=True)
reviewText_train = reviewText_train.reset_index(drop=True)

feature_val = feature_val.reset_index(drop=True)
RRP_x_val = RRP_x_val.reset_index(drop=True)
summary_val = summary_val.reset_index(drop=True)
reviewText_val = reviewText_val.reset_index(drop=True)

feature_test = feature_test.reset_index(drop=True)
testDF = testDF.reset_index(drop=True)
summary_test = summary_test.reset_index(drop=True)
reviewText_test = reviewText_test.reset_index(drop=True)

Wall time: 12.5 s


In [44]:
# %%time
# #TRAIN -------------------------------------------------------
# X_train = pd.concat([feature_train, RRP_x_train[['price','year','sentiment','review_sentiment','summary_sentiment']],summary_train,reviewText_train],axis = 1)
# y_train =  RRP_y_train

# #VALIDATION -------------------------------------------------------
# X_val = pd.concat([feature_val, RRP_x_val[['price','year','sentiment','review_sentiment','summary_sentiment']],summary_val,reviewText_val],axis = 1)
# y_val =  RRP_y_val

# #TEST -------------------------------------------------------
# X_test = pd.concat([feature_test, testDF[['price','year'，'sentiment','review_sentiment','summary_sentiment']],summary_test,reviewText_test],axis = 1)


#TRAIN -------------------------------------------------------
X_train = pd.concat([feature_train,summary_train,reviewText_train],axis = 1)
y_train =  RRP_y_train

#VALIDATION -------------------------------------------------------
X_val = pd.concat([feature_val,summary_val,reviewText_val],axis = 1)
y_val =  RRP_y_val

#TEST -------------------------------------------------------
X_test = pd.concat([feature_test,summary_test,reviewText_test],axis = 1)

In [45]:
# %%time

# del RRP_x_train
# del summary_train
# del feature_train 
# del reviewText_train

# del feature_val
# del RRP_x_val 
# del summary_val
# del reviewText_val 

# del trainDF
# del testDF 
# del feature_test
# del summary_test
# del reviewText_test

# import gc
# gc.collect()

# gc.collect()
# gc.collect()
# gc.collect()
# gc.collect()

In [46]:
# %%time
# #TRAIN -------------------------------------------------------
# X_train = hstack((csr_matrix(feature_train), 
#                      csr_matrix(trainDF[['price','sentiment']]),
#                      csr_matrix(summary_train), 
#                      csr_matrix(reviewText_train)))
# y_train =  RRP_y_train

# #VALIDATION -------------------------------------------------------
# X_val = hstack((csr_matrix(feature_train), 
#                      csr_matrix(trainDF[['price','sentiment']]),
#                      csr_matrix(summary_train), 
#                      csr_matrix(reviewText_train)))
# y_val =  RRP_y_val

# #TEST -------------------------------------------------------
# RRP_x_test = hstack((csr_matrix(feature_test), 
#                      csr_matrix(testDF[['price','sentiment']]),
#                      csr_matrix(summary_test), 
#                      csr_matrix(reviewText_test)))

In [47]:
# %%time
# #TRAIN -------------------------------------------------------
# RRP_x_train = hstack((csr_matrix(feature_train), 
#                      csr_matrix(trainDF[['price','sentiment']]),
#                      csr_matrix(summary_train), 
#                      csr_matrix(reviewText_train)))
# RRP_y_train =  trainDF['overall']

# #TEST -------------------------------------------------------
# RRP_x_test = hstack((csr_matrix(feature_test), 
#                      csr_matrix(testDF[['price','sentiment']]),
#                      csr_matrix(summary_test), 
#                      csr_matrix(reviewText_test)))

### Save cleaned data for future

In [48]:
# testDF.to_csv('testDF.csv')
# trainDF.to_csv('trainDF.csv')

In [49]:
# %%time
# feature_train.to_csv('feature_train.csv')
# summary_train.to_csv('summary_train.csv')
# reviewText_train.to_csv('reviewText_train.csv')

# feature_test.to_csv('feature_test.csv')
# testDF.to_csv('testDF.csv')
# summary_test.to_csv('summary_test.csv')
# reviewText_test.to_csv('reviewText_test.csv')

In [50]:
# %%time
# testDF = pd.read_csv('testDF.csv')
# trainDF = pd.read_csv('trainDF.csv')

## Model Implementation

In [51]:
%%time
# normalize inputs
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

Wall time: 3min 39s
Parser   : 573 ms


In [52]:
# del X_train 
# del X_val
# del X_test

# gc.collect()

# gc.collect()

In [119]:
# # define model
# # MSE on validation set is 0.5067778693405807
# model = Sequential()
# model.add(Dense(64, activation="sigmoid",kernel_regularizer=regularizers.l2(0.0002)))
# model.add(Dropout(0.2))
# model.add(Dense(64, activation="sigmoid",kernel_regularizer=regularizers.l2(0.0002)))
# model.add(Dropout(0.2))
# model.add(Dense(5, activation='softmax'))
# model.add(Dropout(0.1))
# model.add(Dense(1))

best_error = -1
best_params = {}
for lam in [0.0001,0.0002,0.0003]:
    for ep in [10,20,30]:
#         model = Sequential()
#         model.add(Reshape((1,X_train_scaled.shape[1])))
#         model.add(LSTM(80, activation='sigmoid', input_shape=(1, X_train_scaled.shape[1])))
#         model.add(Dense(60, activation="sigmoid",kernel_regularizer=regularizers.l2(lam)))
#         model.add(Dropout(0.2))
#         model.add(Dense(40, activation="sigmoid",kernel_regularizer=regularizers.l2(lam)))
#         model.add(Dropout(0.2))
#         model.add(Dense(20, activation='softmax'))
#         model.add(Dropout(0.1))
#         model.add(Dense(1))
        model = Sequential()
        model.add(Dense(64, activation="sigmoid",kernel_regularizer=regularizers.l2(lam)))
        model.add(Dropout(0.2))
        model.add(Dense(64, activation="sigmoid",kernel_regularizer=regularizers.l2(lam)))
        model.add(Dropout(0.2))
        model.add(Dense(64, activation='sigmoid'))
        model.add(Dropout(0.1))
        model.add(Dense(1))

        # compile network
        model.compile(loss='mse', optimizer='adam', metrics = ['mse'])
        # fit network
        model.fit(X_train_scaled,y_train, epochs = ep)
        # evaluate
        y_pred = model.predict(X_val_scaled)
        print("MSE on validation set is",mean_squared_error(y_val, y_pred))
        
        if best_error == -1:
            best_model = model
            best_param = {'l2':lam,'ep':ep}
            best_error = mean_squared_error(y_val, y_pred)
        if mean_squared_error(y_val, y_pred) < best_error:
            best_model = model
            best_param = {'l2':lam,'ep':ep}

Epoch 1/30
4375/4375 [==============================] - 6s 1ms/step - loss: 0.7792 - mse: 0.7007
Epoch 2/30
4375/4375 [==============================] - 6s 1ms/step - loss: 0.6369 - mse: 0.5374
Epoch 3/30
4375/4375 [==============================] - 6s 1ms/step - loss: 0.6166 - mse: 0.5126
Epoch 4/30
4375/4375 [==============================] - 6s 1ms/step - loss: 0.5986 - mse: 0.4937
Epoch 5/30
4375/4375 [==============================] - 6s 1ms/step - loss: 0.5851 - mse: 0.4789
Epoch 6/30
4375/4375 [==============================] - 6s 1ms/step - loss: 0.5720 - mse: 0.4647
Epoch 7/30
4375/4375 [==============================] - 6s 1ms/step - loss: 0.5628 - mse: 0.4547
Epoch 8/30
4375/4375 [==============================] - 6s 1ms/step - loss: 0.5553 - mse: 0.4459
Epoch 9/30
4375/4375 [==============================] - 6s 1ms/step - loss: 0.5461 - mse: 0.4370
Epoch 10/30
4375/4375 [==============================] - 6s 1ms/step - loss: 0.5410 - mse: 0.4308
Epoch 11/30
4375/4375 [======

In [120]:
# verify mse with best model on validation set
y_pred = best_model.predict(X_val_scaled)
print("MSE on validation set is",mean_squared_error(y_val, y_pred))

MSE on validation set is 0.4747610608281431


In [122]:
y_val

119737    4.0
72272     5.0
158154    5.0
65426     5.0
30074     5.0
         ... 
97771     5.0
59813     5.0
103735    5.0
180226    5.0
119389    5.0
Name: overall, Length: 60000, dtype: float64

In [123]:
y_pred

array([[3.0785718],
       [4.928782 ],
       [4.867382 ],
       ...,
       [4.8526893],
       [4.0465136],
       [4.8157644]], dtype=float32)

## Fit on Test Set

In [124]:
RRP = best_model.predict(X_test_scaled)

### Format prediction

In [125]:
%%time
prediction_list = []
for r in RRP:
    prediction_list.append(r[0])

    
predictions = open('rating_prediction.csv', 'w')
rate = 0
for l in open('rating_pairs.csv'):
    if l.startswith('userID'):
        #header
        predictions.write(l)
        continue
    u,p = l.strip().split('-')
    predictions.write(u + '-' + p +  ',' + str(prediction_list[rate]) + '\n')
    rate += 1

Wall time: 103 ms


In [127]:
RRP

array([[4.6583834],
       [4.786703 ],
       [4.8100166],
       ...,
       [4.726027 ],
       [2.961877 ],
       [4.8356757]], dtype=float32)

In [ ]:
# kaggle score 0.49979